In [76]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from math import sqrt

In [77]:
data = pd.read_csv('/home/ayarkhan01/Desktop/Projects/stock_data_final.csv')

In [79]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

/tmp/ipykernel_24837/889507750.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['Date'] = pd.to_datetime(data['Date'])


In [85]:
# Assuming 'data' is your DataFrame
data = data.dropna()

In [87]:
def create_sequences(input_data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(input_data) - n_steps_in - n_steps_out + 1):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        # gather input and output parts of the pattern
        seq_x = input_data[i:end_ix, :]  # Use all available features for input
        seq_y = input_data[end_ix:out_end_ix + 1, 2]  # Use the 'Close' price for output
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [88]:
# Convert DataFrame to numpy array for processing
data_array = data.to_numpy()

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_array)

n_steps_in, n_steps_out = 60, 1  # Number of timesteps in / out
X, y = create_sequences(scaled_data, n_steps_in, n_steps_out)

tscv = TimeSeriesSplit(n_splits=5)

In [89]:
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # LSTM Model
    model = Sequential()
    model.add(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(LSTM(64, activation='tanh', return_sequences=False))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Fit model
    model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)
    
    # Evaluate
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test.flatten(), y_pred.flatten()))
    print(f"RMSE: {rmse}")

Epoch 1/100


/home/ayarkhan01/Desktop/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0025
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 8.1364e-04
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 1.5626e-04
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 9.6945e-05
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 2.4629e-05
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 2.2665e-05
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 1.2320e-05
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 8.6073e-06
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 8.2233e-06
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 6.1218e-06
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 6.2276e-06
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 6.2021e-06
Epoch 13/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 5.7514e-06
Epoch 14/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 5.3930e-06
Epoch 15/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms

/home/ayarkhan01/Desktop/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: 0.0014
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 1.1152e-04
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 3.8365e-05
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 1.8752e-05
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 1.0061e-05
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 7.5171e-06
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 6.9603e-06
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 7.3284e-06
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 6.8218e-06
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 7.0061e-06
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 6.6476e-06
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 6.6960e-06
Epoch 13/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 6.7325e-06
Epoch 14/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 6.9332e-06
Epoch 15/100
18/1

...

26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 0.0096
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 2.7060e-04
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 6.5607e-05
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 3.7860e-05
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 2.8572e-05
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 2.3548e-05
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 1.9580e-05
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 1.6322e-05
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 1.4224e-05
Epoch 10/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 1.3401e-05
Epoch 11/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 1.2718e-05
Epoch 12/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 1.2498e-05
Epoch 13/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 1.2097e-05
Epoch 14/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 1.1321e-05
Epoch 15/100
26/2

/home/ayarkhan01/Desktop/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.0028
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 6.7770e-05
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 4.1427e-05
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 3.1466e-05
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 2.5657e-05
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 2.3314e-05
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 2.1855e-05
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 2.4767e-05
Epoch 9/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 1.9707e-05
Epoch 10/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 1.9437e-05
Epoch 11/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 2.0448e-05
Epoch 12/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 2.0870e-05
Epoch 13/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 1.9240e-05
Epoch 14/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 1.8383e-05
Epoch 15/100
35/3

/home/ayarkhan01/Desktop/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: 0.0166
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 2.5923e-04
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 1.7121e-04
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 1.3939e-04
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 1.2100e-04
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.2079e-04
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 1.2143e-04
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 1.1865e-04
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.1680e-04
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 1.3382e-04
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 1.1022e-04
Epoch 12/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.1870e-04
Epoch 13/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 9.4432e-05
Epoch 14/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 9.5641e-05
Epoch 15/100
43/4

In [90]:
model.save('model_name.keras')